In [ ]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x_train = pd.read_csv(Config.train_path)
x_train.head()

In [ ]:
x_train.shape

In [ ]:
x_train.columns

In [ ]:
tmp = x_train[x_train['adr'] < 5000]
tmp = tmp[(tmp['adults']+tmp['children']+tmp['babies']) < 10]
plt.scatter(tmp['adults']+tmp['children']+tmp['babies'], tmp['adr'], alpha=.1)

In [ ]:
x_train[x_train['adr'] > 5000]

In [ ]:
from HTML.preprocessing import preprocessing
x_train = preprocessing(x_train)
x_train.head()

In [ ]:
from sklearn.feature_selection import chi2

In [ ]:
x_train.head()[{'reservation_status', 'reservation_status_date', 'is_canceled'}]

In [ ]:
dummies = pd.get_dummies(
        data=x_train[set(x_train.columns) - {'reservation_status', 'reservation_status_date', 'is_canceled'}],
        columns=set(nominal+ordinal) - {'reservation_status', 'reservation_status_date', 'is_canceled'},
        drop_first=True,
        dummy_na=True)
dummies = dummies[sorted(set(dummies.columns) - {'index', 'ID'} - set(meaningless))]

In [ ]:
x = dummies[sorted(set(dummies.columns) - {'adr'})].to_numpy()
y = dummies['adr'].to_numpy()

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [ ]:
# x_new = SelectKBest(f_regression, k=3).fit_transform(x, y)

In [ ]:
x.shape

In [ ]:
# x_new.shape

In [ ]:
# from sklearn.preprocessing import Normalizer
# transformer = Normalizer()
# x_norm = transformer.fit_transform(x)

In [ ]:
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

In [ ]:
dummies_sm = pd.get_dummies(
        data=x_train[x_train['adr'] < 5000][set(x_train.columns) - {'adr', 'reservation_status', 'reservation_status_date', 'is_canceled'}],
        columns=set(nominal+ordinal) - {'adr', 'reservation_status', 'reservation_status_date', 'is_canceled'},
        drop_first=True,
        dummy_na=True)
dummies_sm = dummies_sm[sorted(set(dummies_sm.columns) - {'index', 'ID'} - set(meaningless))]
x_sm = dummies_sm[sorted(set(dummies_sm.columns))].to_numpy()
y_sm = x_train[x_train['adr'] < 5000]['adr'].to_numpy()

In [ ]:
x_sm.shape

In [ ]:
y_sm.shape

In [ ]:
# s = []
# for i in [0,1,3,5,10]:
#     regr = XGBRegressor(
#         n_estimators=100,
#         learning_rate=.3,
#         max_depth=6,
#         subsample=1,
#         n_jobs=6,
#         colsample_bytree=.7,
#         random_state=1126,
#         gamma=i
#     )
#     scores = cross_val_score(regr, x_sm, y_sm, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=2)
#     s.append(scores)
#     print(np.mean(scores))

In [ ]:
# plt.plot([0,1,3,5,10], np.mean(np.array(s),axis=1))

In [ ]:
# regr = SVR()
# regr = ElasticNetCV(cv=5, random_state=0, n_jobs=-1)
# regr = RandomForestRegressor(n_jobs=-1)
# regr = SGDRegressor()
regr = XGBRegressor(
    n_estimators=100,
    learning_rate=.3,
    max_depth=6,
    subsample=1,
    n_jobs=6,
    colsample_bytree=.7,
    random_state=1126,
    gamma=10
)

In [ ]:
regr

In [ ]:
regr.fit(x_sm, y_sm)

In [ ]:
pred = regr.predict(x_sm)

In [ ]:
plt.scatter(pred, y_sm, alpha=.1)

In [ ]:
scores = cross_val_score(regr, x_sm, y_sm, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=2)

In [ ]:
scores

In [ ]:
np.mean(scores)

In [ ]:
x_train['adr'].max()

In [ ]:
x_train['adr'].min()

In [ ]:
x_train['adr'].mean()

In [ ]:
np.mean(np.sqrt(-scores))